In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool, Tool
from langchain.prompts import PromptTemplate

#Clientes de chat para el selector de LLM 
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_google_genai import ChatGoogleGenerativeAI

# Importamos nuestras herramientas personalizadas desde el directorio padre
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.tools.calculadora import calcular_promedio_de_notas
from src.tools.buscador import buscar_en_faq, buscador_de_reglamentos

# Cargar variables de entorno
load_dotenv()

# Módulo de Reglas de Seguridad
def filtro_de_seguridad(consulta: str) -> bool:
    """
    Filtro simple para detectar consultas inapropiadas.
    Devuelve True si la consulta es segura, False si no lo es.
    """
    consultas_prohibidas = ["examen", "respuestas", "dame la prueba", "hackear"]
    for palabra in consultas_prohibidas:
        if palabra in consulta.lower():
            return False
    return True

#Definición de Herramientas para LangChain
@tool
def calculadora_academica(consulta: str) -> str:
    """Útil para cuando el usuario pide calcular el promedio de varias notas."""
    return calcular_promedio_de_notas(consulta)

@tool
def buscador_faq(consulta: str) -> str:
    """Útil para responder preguntas frecuentes sobre temas generales como horarios o correos."""
    return buscar_en_faq(consulta)

# Creación del Agente con Selector de LLM
def crear_agente():
    """
    Configura y crea el agente conversacional con selector de proveedor LLM.
    """
    herramienta_buscador_reglamentos = Tool(
        name="buscador_reglamentos",
        func=buscador_de_reglamentos.buscar,
        description="Útil para responder preguntas específicas sobre el reglamento académico, como reglas de asistencia, calificaciones, o condiciones de examen."
    )
    tools = [calculadora_academica, buscador_faq, herramienta_buscador_reglamentos]

    #Selector LLM
    # Opciones: "GOOGLE", "HUGGINGFACE"
    LLM_PROVIDER = "GOOGLE"  # <-- Cambia aquí para elegir el proveedor

    print(f"--- Usando el proveedor de LLM: {LLM_PROVIDER} ---")

    if LLM_PROVIDER == "GOOGLE":
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)
        
    elif LLM_PROVIDER == "HUGGINGFACE":
        llm_endpoint = HuggingFaceEndpoint(
            repo_id="mistralai/Mistral-7B-Instruct-v0.2",
            task="text-generation",
            max_new_tokens=256,
            temperature=0.1,
            stop_sequences=["\nObservation:", "\nThought:"]
        )
        llm = ChatHuggingFace(llm=llm_endpoint)
    else:
        raise ValueError(f"Proveedor '{LLM_PROVIDER}' no reconocido. Opciones: GOOGLE, HUGGINGFACE")

    #Prompt
    template = """
    Eres un asistente que responde preguntas usando herramientas. Sigue estas reglas ESTRICTAMENTE.
    Para ayudarte, tienes acceso a las siguientes herramientas:
    {tools}
    REGLAS DE FORMATO:
    1.  Tu primer paso DEBE seguir este formato:
        Question: La pregunta que debes responder.
        Thought: Tu análisis de la pregunta y qué herramienta usar.
        Action: La herramienta a usar, debe ser una de [{tool_names}].
        Action Input: La entrada para la herramienta.
    2.  Después de `Action Input`, DETENTE. No escribas nada más. El sistema te dará la `Observation`.
    3.  Después de recibir una `Observation`, si ya tienes la respuesta, DEBES usar este formato:
        Thought: Ya tengo la respuesta final.
        Final Answer: Tu respuesta final a la pregunta del usuario.
    4.  REGLA CRÍTICA: Si la `Observation` indica que no se encontró la información, tu respuesta final DEBE ser que no pudiste encontrar la información. No inventes respuestas.
    --- EJEMPLO DE USO ---
    Question: ¿cuál es el promedio de 5 y 4?
    Thought: El usuario quiere calcular un promedio. Debo usar la herramienta 'calculadora_academica'.
    Action: calculadora_academica
    Action Input: 5, 4
    --- FIN DEL EJEMPLO ---
    Ahora, empieza.
    Question: {input}
    {agent_scratchpad}
    """

    prompt = PromptTemplate.from_template(template)
    agent = create_react_agent(llm, tools, prompt)
    
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=5,
    )
    return agent_executor

print("✓ Agente configurado correctamente con selector de LLM")

c:\Users\Delii\OneDrive\Documentos\Programacion\TP final IA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Agente configurado correctamente con selector de LLM


In [3]:
from src.tools.calculadora import calcular_promedio_de_notas

In [4]:
pregunta1 = "¿cuál es el promedio entre 4.0, 5.5 y 7.0?"
print(calcular_promedio_de_notas(pregunta1))

El promedio de las notas [4.0, 5.5, 7.0] es: 5.50


In [5]:
pregunta2 = "necesito la media de 3,5 y un 2"
print(calcular_promedio_de_notas(pregunta2))

El promedio de las notas [3.0, 5.0, 2.0] es: 3.33


In [6]:
pregunta3 = "hola cómo estás"
print(calcular_promedio_de_notas(pregunta3))


No encontré números en tu pregunta para calcular el promedio.


## Prueba del Agente Completo

Ahora vamos a probar el agente con el selector de LLM

In [7]:
# Crear el agente
agente = crear_agente()

# Probar con una pregunta sobre calculadora
pregunta_agente = "¿Cuál es el promedio de 8, 9 y 7?"
print(f"\n{'='*60}")
print(f"Pregunta: {pregunta_agente}")
print(f"{'='*60}\n")

respuesta = agente.invoke({"input": pregunta_agente})
print(f"\n{'='*60}")
print(f"Respuesta Final: {respuesta['output']}")
print(f"{'='*60}")

--- Usando el proveedor de LLM: GOOGLE ---

Pregunta: ¿Cuál es el promedio de 8, 9 y 7?



> Entering new AgentExecutor chain...
Question: ¿Cuál es el promedio de 8, 9 y 7?
Thought: El usuario quiere calcular un promedio de varias notas. Debo usar la herramienta 'calculadora_academica'.
Action: calculadora_academica
Action Input: 8, 9, 7El promedio de las notas [8.0, 9.0, 7.0] es: 8.00Question: ¿Cuál es el promedio de 8, 9 y 7?
Thought: El usuario quiere calcular un promedio de varias notas. Debo usar la herramienta 'calculadora_academica'.
Action: calculadora_academica
Action Input: 8, 9, 7El promedio de las notas [8.0, 9.0, 7.0] es: 8.00Ya tengo la respuesta final.
Final Answer: El promedio de 8, 9 y 7 es 8.00.

> Finished chain.

Respuesta Final: El promedio de 8, 9 y 7 es 8.00.
Ya tengo la respuesta final.
Final Answer: El promedio de 8, 9 y 7 es 8.00.

> Finished chain.

Respuesta Final: El promedio de 8, 9 y 7 es 8.00.
